In [13]:
import pandas as pd
import nltk
from textblob import Word
# import pickle
import re
import string
from sklearn.feature_extraction.text import CountVectorizer
from gensim import matutils, models
import scipy.sparse
from sklearn.feature_extraction import text
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from sklearn.pipeline import Pipeline
from sklearn.ensemble import ExtraTreesClassifier
# from wordcloud import WordCloud
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to C:\Users\The Black
[nltk_data]     Hawk\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\The Black
[nltk_data]     Hawk\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\The Black
[nltk_data]     Hawk\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [14]:
def clean_text_round1(text):
    text = str(text.lower())
    text = re.sub('http.* ', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    return text


In [15]:
train = pd.read_csv("nlp-getting-started/train.csv")
test = pd.read_csv("nlp-getting-started/test.csv")
cv = CountVectorizer(stop_words='english')


print(train.head())

   id keyword location                                               text  \
0   1     NaN      NaN  Our Deeds are the Reason [][][][][][of this #e...   
1   4     NaN      NaN             Forest fire near La Ronge Sask. Canada   
2   5     NaN      NaN  All residents asked to 'shelter in place' are ...   
3   6     NaN      NaN  13,000 people receive #wildfires evacuation or...   
4   7     NaN      NaN  Just got sent this photo from Ruby #Alaska as ...   

   target  
0       1  
1       1  
2       1  
3       1  
4       1  


In [16]:
round1 = lambda x: clean_text_round1(x) 
print(train.text)
train.text = pd.DataFrame(train.text.apply(round1))
test.text = pd.DataFrame(test.text.apply(round1))
list(train.columns.values) 
print(train.text)

0       Our Deeds are the Reason [][][][][][of this #e...
1                  Forest fire near La Ronge Sask. Canada
2       All residents asked to 'shelter in place' are ...
3       13,000 people receive #wildfires evacuation or...
4       Just got sent this photo from Ruby #Alaska as ...
                              ...                        
7608    Two giant cranes holding a bridge collapse int...
7609    @aria_ahrary @TheTawniest The out of control w...
7610    M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...
7611    Police investigating after an e-bike collided ...
7612    The Latest: More Homes Razed by Northern Calif...
Name: text, Length: 7613, dtype: object
0       our deeds are the reason of this earthquake ma...
1                   forest fire near la ronge sask canada
2       all residents asked to shelter in place are be...
3        people receive wildfires evacuation orders in...
4       just got sent this photo from ruby alaska as s...
                              ..

In [36]:
train_target = train["target"]
train_data = train.drop("target", axis=1)
# test_target = test["target"]
# test_data = test.drop("target", axis=1)
stop = stopwords.words('english')
stop.append('u')
# print(stop)

# ----------APPLY DATA CLEANING
train_data['text'] = train_data['text'].apply(lambda x:' '.join(x for x in x.split() if not x in stop))
# test_data['text'] = test_data['text'].apply(lambda x:' '.join(x for x in x.split() if not x in stop))

# train_data['text'] = train_data['text'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
print(train_data['text'].head())
# train_data1=train_data[['id','text']]
# token_data=[]
# for i in train_data['text']:
#     token_data.append(nltk.word_tokenize(i))
# print(token_data)
bigger_list=[]
for i in train_data['text']:
    li = list(i.split(" "))
    bigger_list.append(li)
# for i in test_data['text']:
#     li = list(i.split(" "))
#     bigger_list.append(li)
#structure of data to be taken by the model.word2vec
# print("Data format for the overall list:",bigger_list)
#custom data is fed to machine for further processing
model = Word2Vec(bigger_list, min_count=1,size=300,workers=4)
w2v = dict(zip(model.wv.index2word, model.wv.vectors))
print(model)
# print(w2v)
print("******")


0         deeds reason earthquake may allah forgive us
1                forest fire near la ronge sask canada
2    residents asked shelter place notified officer...
3    people receive wildfires evacuation orders cal...
4    got sent photo ruby alaska smoke wildfires pou...
Name: text, dtype: object
Word2Vec(vocab=16915, size=300, alpha=0.025)
******


In [37]:
 print(list(model.wv.vocab) )

, 'colorectal', 'humanityi', 'apologized', 'survivorsr', 'sojapan', 'nuclearweapons', 'coppednews', 'haunting', 'kotowsa', 'sudan\x89ûªs', 'mithitennis', 'crackedgem', 'httptcoggnbvzohwu', 'molecularly', 'targeted', 'lungcancer', 'violators', 'improved', 'hurled', 'nuked', 'nears', 'focuses', 'fedporn', 'absurd', 'satire', 'indistinguishable', 'reality', '\x89û÷faceless', 'sister\x89ûª', 'shanghai', 'ghetto', 'reunite', 'netflix', 'documentary', 'narrated', 'parter', 'interviews', 'lawfulsurvivor', 'tdog', 'holed', 'glenn', 'morales', 'andrea', 'jacqui', 'merle', 'stemming', 'cubs', 'rosters', 'haley', 'lu', 'richardson', 'httptcooobscfoktq', 'remembrance', 'truthnewsbbccnnislamtruthgodisisterrorismquranlies', 'islamist', 'anjem', 'choudary', 'robpulsenews', 'huyovoetripolye', 'phillips', 'ldnrterrorists', 'httptcoudkmadkuzy', 'preacher', 'faces', 'httptcodaplllfuqk', 'sneaks', 'cockpit', 'democracy\x89ûªs', 'dawabsha', 'erode', 'homegrown', 'truthhttpstcobejftygjilnewsbbccnnislamtruth

In [38]:
similar_words = model.wv.most_similar('fart')	
print(similar_words)	

[('sins', 0.2796197831630707), ('httptcognzwxqktam', 0.25066015124320984), ('printing', 0.22455847263336182), ('kombat', 0.22090771794319153), ('system', 0.21606189012527466), ('overnight', 0.21453416347503662), ('displacedinjuredkilled', 0.2123355269432068), ('throwin', 0.20703837275505066), ('credited', 0.20587119460105896), ('test', 0.20249271392822266)]


In [25]:
print(model['disaster'])

[ 3.43667064e-03  1.23436050e-02 -6.69078249e-03 -7.61220930e-04
  8.77864379e-03 -1.75273861e-03  2.55663018e-03  8.87596700e-03
  6.47313776e-04  1.01737157e-02 -5.16054127e-03  1.27925130e-03
  1.83120044e-03 -6.74367882e-03 -2.77801929e-03  4.47372114e-03
  5.74486051e-03  1.97366392e-03  6.83540478e-03 -6.95012044e-03
  1.82959605e-02 -6.38897438e-03  6.08488172e-03  2.72689480e-03
  6.78411219e-03  2.06522527e-03 -4.39010700e-03 -1.48089239e-02
  1.70867308e-03  2.50097783e-03 -1.23223639e-03  5.73651865e-03
 -1.40677765e-03 -3.60137480e-03 -1.17303543e-02 -3.74244736e-03
  5.90924174e-03  3.31979623e-04 -7.44515099e-04 -7.16887973e-03
  3.78438574e-03  7.03256158e-03 -1.01774279e-02 -6.36471494e-04
 -4.63861553e-03 -1.48901320e-03  4.72006062e-03  3.22265667e-03
  2.73729005e-04  1.04394287e-03  1.32095292e-02  2.72798329e-03
  8.70610215e-03  1.44086434e-02 -1.41716341e-03  1.06589627e-02
  6.32976647e-03 -1.59420935e-03  9.78029799e-03  7.79368589e-03
 -2.52584112e-03 -2.73679

In [21]:
print(1+2)

3


In [22]:
docs_vectors = pd.DataFrame()
print("ds")
for i in train_data['text']:
    li = list(i.split(" "))
    temp = pd.DataFrame()
    for word in li:
        word_vec = model.wv[word]
        temp = temp.append(pd.Series(word_vec), ignore_index = True)
    doc_vector = temp.mean()
    docs_vectors = docs_vectors.append(doc_vector, ignore_index = True)
print(docs_vectors.shape)
print("************************")
docs_vectors_test = pd.DataFrame()
print("ds")
for i in train_data['text']:
    li = list(i.split(" "))
    temp = pd.DataFrame()
    for word in li:
        word_vec = model.wv[word]
        temp = temp.append(pd.Series(word_vec), ignore_index = True)
    doc_vector = temp.mean()
    docs_vectors_test = docs_vectors.append(doc_vector, ignore_index = True)
print(docs_vectors_test.shape)
print("************************")

ds
(7613, 300)
************************
ds
(7614, 300)
************************


In [26]:
print(docs_vectors.shape)
print("dss")

(7613, 300)
dss


In [27]:


from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
train_x, test_x, train_y, test_y = train_test_split(docs_vectors,
                                                   train_target,
                                                   test_size = 0.2,
                                                   random_state = 1)
train_x.shape, train_y.shape, test_x.shape, test_y.shape

((6090, 300), (6090,), (1523, 300), (1523,))

In [28]:
model = AdaBoostClassifier(n_estimators=800, random_state = 1)
model.fit(train_x, train_y)
print("done")

done


In [ ]:
print("something")
test_pred = model.predict(test_x)
from sklearn.metrics import accuracy_score
accuracy_score(test_y, test_pred)

print(accuracy_score(test_y, test_pred))
print("something 2")


In [21]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=800, random_state = 1)
model.fit(train_x, train_y)
print("done")
print("something")
test_pred = model.predict(test_x)
from sklearn.metrics import accuracy_score
accuracy_score(test_y, test_pred)

print(accuracy_score(test_y, test_pred))
print("something 2")

done
something
0.7353906762967827
something 2


In [24]:
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
#     GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1, max_iter=1000),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()]
names = ["Nearest Neighbors", "Linear SVM", "RBF SVM",
#          "Gaussian Process",
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
         "Naive Bayes", "QDA"]
for i in range(len(classifiers)):
    model = classifiers[i]
    model.fit(train_x, train_y)
    print("done")
    print("something")
    test_pred = model.predict(test_x)
    from sklearn.metrics import accuracy_score
    accuracy_score(test_y, test_pred)

    print(names[i]," accuracy: ",accuracy_score(test_y, test_pred))
    print("something 2")

done
something
Nearest Neighbors  accuracy:  0.6940249507550886
something 2
done
something
Linear SVM  accuracy:  0.5791201575837164
something 2
done
something
RBF SVM  accuracy:  0.5791201575837164
something 2
done
something
Decision Tree  accuracy:  0.6073539067629679
something 2
done
something
Random Forest  accuracy:  0.602757715036113
something 2
done
something
Neural Net  accuracy:  0.5791201575837164
something 2
done
something
AdaBoost  accuracy:  0.6559422193040052
something 2
done
something
Naive Bayes  accuracy:  0.5436638214051215
something 2
done
something
QDA  accuracy:  0.7518056467498359
something 2
